# 1. Install depedencies

In [1]:
!python --version

Python 3.9.23


In [8]:
!python -c "import torch; import torchvision; import sys;print('\nIgnore output above, BELOW ARE VERSION LISTED:\n');print('Python version:',sys.version);print('Pytorch version:', torch.__version__); print('Torchvision version:', torchvision.__version__)"

/opt/anaconda3/envs/uzslr-signs/lib/python3.9/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/uzslr-signs/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <FB2FD416-6C4D-3621-B677-61F07C02A3C5> /opt/anaconda3/envs/uzslr-signs/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/uzslr-signs/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/uzslr-signs/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/uzslr-signs/lib/python3.9/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/uzslr-signs/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environ

In [30]:
!conda export --from-history

name: uzslr-signs
channels:
  - conda-forge
  - defaults
  - https://repo.anaconda.com/pkgs/main
  - https://repo.anaconda.com/pkgs/r
dependencies:
  - python=3.9.23
  - ipykernel
  - pytorch=2.5.1
  - torchvision=0.20.1
  - jupyter
  - pandas
  - matplotlib
  - numpy
  - scikit-learn
  - tqdm
  - opencv
  - pathlib
prefix: /opt/anaconda3/envs/uzslr-signs


In [3]:
!which pip

/opt/anaconda3/envs/uzslr-signs/bin/pip


In [4]:
!which pip3

/opt/anaconda3/envs/uzslr-signs/bin/pip3


In [9]:
#!conda install pytorch=2.5.1 torchvision=0.20.1 torchaudio -c pytorch

In [10]:
#!pip3/conda uninstall torch torchvision torchaudio -y # delete pytorch

In [17]:
!conda install -q --yes numpy pandas matplotlib scikit-learn tqdm pathlib

Channels:
 - conda-forge
 - defaults
 - pytorch
Platform: osx-arm64
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/anaconda3/envs/uzslr-signs

  added / updated specs:
    - matplotlib
    - numpy
    - pandas
    - scikit-learn
    - tqdm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    colorama-0.4.6             |     pyhd8ed1ab_1          26 KB  conda-forge
    fsspec-2025.7.0            |     pyhd8ed1ab_0         142 KB  conda-forge
    libabseil-20250127.1       | cxx17_h07bc746_0         1.1 MB  conda-forge
    libprotobuf-5.29.3         |       h7463a21_3         2.5 MB  conda-forge
    libtorch-2.5.1             |cpu_generic_h0f11145_17        27.0 MB  conda-forge
    libuv-1.51.0               |       h6caf38d_1         411 KB  conda-forge
    nomkl-1.0                  |       h5ca1d4c_0           4 KB  conda-forge
    pybind11-3.0.1 

In [18]:
!conda install -q --yes -c conda-forge opencv

Channels:
 - conda-forge
 - defaults
 - pytorch
Platform: osx-arm64
Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/anaconda3/envs/uzslr-signs

  added / updated specs:
    - opencv


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aom-3.9.1                  |       h7bae524_0         2.1 MB  conda-forge
    c-ares-1.34.6              |       hc919400_0         176 KB  conda-forge
    cyrus-sasl-2.1.28          |       ha1cbb27_0         189 KB  conda-forge
    dav1d-1.2.1                |       hb547adb_0         309 KB  conda-forge
    dbus-1.16.2                |       h3ff7a7c_1         385 KB  conda-forge
    double-conversion-3.3.1    |       h286801f_0          62 KB  conda-forge
    ffmpeg-7.1.1               | gpl_h20db955_106         8.7 MB  conda-forge
    hdf5-1.14.6                |nompi_hd3baa01_104         3.1 MB  conda-forge
    i

# 2. Config

In [6]:
# checking if mps is enabled. MPS is Apple Sillion GPU
!python -c "import torch; print(f'MPS available: {torch.backends.mps.is_available()}'); print(f'MPS built with PyTorch: {torch.backends.mps.is_built()}')"


MPS available: True
MPS built with PyTorch: True


In [24]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import matplotlib as mpl

from tqdm.autonotebook import tqdm
import sklearn

import os
import time
import pickle
import math
import random
import sys
import cv2
import gc
import glob
import datetime
from pathlib import Path

print(f'PyTorch Version: {torch.__version__}')
print(f'Python Version: {sys.version}')

PyTorch Version: 2.5.1
Python Version: 3.9.23 | packaged by conda-forge | (main, Jun  4 2025, 18:02:02) 
[Clang 18.1.8 ]


# 3. Work

In [21]:
def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def get_device():
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        device_name = "Apple MPS"
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        device_name = "CUDA GPU"
    else:
        device = torch.device("cpu")
        device_name = "CPU"

    print(f"Using device: {device_name}")
    return device

In [22]:
seed_everything(42)
device = get_device()
N_REPLICAS = 1

Using device: Apple MPS


## 3.1 loading dataset

In [25]:
DATA_ROOT = Path("../../data")
SPLITS = ["train", "validation", "test"]

# 50 signs
DEFAULT_SIGNS = ['assalomu_alaykum', 'bahor', 'birga', "bo'sh", 'bosh_kiyim', 'boshlanishi', 'bozor', 'eshik', 
               'futbol', 'iltimos', 'internet', 'javob', 'jismoniy_tarbiya', 'karam', 'kartoshka', 
               'kichik', 'kitob', "ko'prik", 'likopcha', 'maktab', 'mehmonxona', 'mehribon', 'metro', 
               'musiqa', "o'simlik_yog'i", "o'ynash", 'ochish', 'ot', 'ovqat_tayyorlash', 
               'oxiri', 'poezd', 'pomidor', 'qidirish', 'qish', "qo'ziqorin", 'qor', "qorong'i", 'quyon', 
               'restoran', "sariyog'", 'shokolad', 'sovun', 'stakan', 'televizor', 'tosh', 'toza',
               'turish', "yomg'ir", 'yopish', 'yordam_berish'] 

In [26]:
label2id = {sign: idx for idx, sign in enumerate(DEFAULT_SIGNS)}
id2label = {idx: sign for sign, idx in label2id.items()}

NUM_CLASSES = len(DEFAULT_SIGNS)
print("Number of classes:", NUM_CLASSES)

Number of classes: 50


In [27]:
def build_index(split):
    split_path = DATA_ROOT / split
    samples = []

    for sign in DEFAULT_SIGNS:
        sign_path = split_path / sign
        if not sign_path.exists():
            continue

        label = label2id[sign]

        for rep_dir in sign_path.iterdir():
            if not rep_dir.is_dir():
                continue

            frames = sorted(rep_dir.glob("frame-*.npy"))
            if len(frames) != 32:
                continue  # safety

            samples.append({
                "frames": frames,
                "label": label,
                "sign": sign,
                "rep_path": rep_dir
            })

    return samples

In [29]:
train_samples = build_index("train")
val_samples   = build_index("validation")
test_samples  = build_index("test")

print("Train samples:", len(train_samples))
print("Val samples:", len(val_samples))
print("Test samples:", len(test_samples))

Train samples: 2038
Val samples: 250
Test samples: 300
